# Intro to `tf.keras` for image classification

We'll use the `keras` API included in `tensorflow` to create a model that predicts velocity curves from synthetic seismic images. The dataset comes from a collection of synthetic seismic samples put together by Lukas Mosser and released on Github as [SNIST](https://github.com/LukasMosser/SNIST). This dataset intends to be a standard benchmark similiar to MNIST. We'll look at a basic model first.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
urls = [
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/train/train_amplitudes.npy',
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/train/train_velocities.npy',
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/test/test_amplitudes.npy',
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/test/test_velocities.npy',
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/test/test_amplitudes_noise_1.npy',
        'https://raw.githubusercontent.com/LukasMosser/SNIST/master/data/test/test_amplitudes_noise_2.npy'
    ]

Numpy allows you to point at URL data sources. It'll take care of downloading them and keeping reference of where they are with respect to a root folder specified by the user.

In [ ]:
ds = np.DataSource('../data/')

train_amplitudes = np.load(ds.open(urls[0], mode='rb'))
train_velocities = np.load(ds.open(urls[1], mode='rb'))

In [ ]:
plt.imshow(train_amplitudes[0], aspect=0.06)

In [ ]:
plt.plot(train_velocities[0])

In [ ]:
N_train = 600 #Number of total training examples
N_val = 150 #Number of samples used for validation
N_samples = 271 #Number of samples in time
N_recorders = 20 #Number of recording stations
N_layers = 9 #Number of layers in the target velocity model
N_z = 360 #Number of grid blocks in z-dimension (only used for visualisation)

lr = 1e-2 #Learning rate
batch_size = N_train-N_val #Batchsize used in training - do full batch evaluation because of small data
N_epochs = 200 #Number of epochs to train for

We'll use Tensorflow's Keras functionality to setup a single layer network as baseline.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
tf.version.VERSION

Let's define a single layer network using `tf.keras.models` and `tf.keras.layers`.

In [ ]:
model = tf.keras.models.Sequential()

model.add(Dense(N_recorders*N_samples, input_dim=N_recorders*N_samples, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(N_layers))

# Let's define an optimizer
opt = SGD(lr=1e-2)

model.compile(loss='mse',
              optimizer=opt,
              metrics=['mae'])

We'll need to standarize the input and normalize the output values

In [ ]:
train_mean, train_std = train_amplitudes.mean(), train_amplitudes.std()
train_vel_max = train_velocities.max()

In [ ]:
X = train_amplitudes - train_mean
X /= train_std

y = train_velocities / train_vel_max

X_train = X[:-N_val]
y_train = y[:-N_val]


X_test = X[N_train-N_val:]
y_test = y[N_train-N_val:]

Let's confirm that the shapes of these matrices are OK:

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

We want to make the training data to be 1D to be allowed through this fully connected neural network.

In [ ]:
X_train = X_train.reshape((N_train-N_val, N_samples*N_recorders))
X_test = X_test.reshape((N_val, N_samples*N_recorders))

In theory we could just do `model.fit(X_train, y_train)` and be done with it. But it's useful to setup a log system and callbacks to monitor the progress and quality of the training run.

In [ ]:
curr_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(
    "logs",
    "fit",
    curr_date,
)

model_name = 'seis_vel'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(f'{log_dir}/{model_name}.h5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(X_train, y_train,
          epochs=10,
          batch_size=N_train,
          validation_data=(X_test, y_test),
          callbacks=[tensorboard_callback,
                     es_callback,
                     checkpoint_callback,
                    ],
         )

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.gcf().set_size_inches(15,10)

What if we want to load a pre-trained model?

In [ ]:
model.load_weights(f'logs/fit/{curr_date}/seis_vel.h5')
y_pred = model.predict(X_test[0].reshape(1,*X_test[0].shape))

In [ ]:
plt.plot(y_pred[0])
plt.plot(y_test[0])

<div class="alert alert-success">
Create another model with more layers and more neurons per layer and train it using the same training data. Is it any better?
</div>

In [ ]:
# put your code here

# How about a Convolutional Neural Network?

In [ ]:
model = tf.keras.Sequential()# Must define the input shape in the first layer of the neural network

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(N_samples, N_recorders, 1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_layers))

model.summary()

In [ ]:
model.compile(loss='mse',
              optimizer=opt,
              metrics=['mae'])

<div class="alert alert-success">
To take full advantage of a convolutional neural network, the training data has to maintain its spatial relationships. In this case, the 2D images shouldn't be serialized. When this is the case, `keras` requires the training images to be in a very specific shape: (n_image, width, height, n_channels).

**Exercise:**
* Modify the shape of the X array to match the expected shape of the dataset. `X_train` should have the following shape afterwards: (450, 271, 20, 1).
</div>

In [ ]:
X_train.shape

In [ ]:
curr_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(
    "logs_CNN",
    "fit",
    curr_date,
)

model_name = 'seis_vel_CNN'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(f'{log_dir}/{model_name}.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')


model.fit(X_train, y_train,
          epochs=10,
          batch_size=N_train,
          validation_data=(X_test, y_test),
          callbacks=[tensorboard_callback,
                     es_callback,
                     checkpoint_callback,
                    ],
         )

In [ ]:
y_pred = model.predict(X_test[0].reshape(1,*X_test[0].shape))

In [ ]:
plt.plot(y_pred[0])
plt.plot(y_test[0])

# The fossil dataset
Let's generate a workflow to classify images using a CNN.
We'll make use of a collection of functions in `utils.py` to help process the images found in the `data/fossils` folder.

In [ ]:
import numpy as np
from utils import make_train_test, preprocess_images

In [ ]:
X = np.load('../data/fossils/X.npy')
y = np.load('../data/fossils/y.npy')

Or we can generate a training dataset from the image files themselves

In [ ]:
X, X_val, y, y_val = make_train_test(path='../data/fossils/prod/*/*/*.jpg', skip=['plants'])

In [ ]:
X.shape

We have 528 images of size 32x32. Let's look at one:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(X[2].reshape(32,32))
plt.colorbar()

### Shallow learning

We can always use a shallow learning model on our dataset. This will give us something to beat with a neural network.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

clf = RandomForestClassifier(n_estimators=100)

clf.fit(X, y)
y_pred = clf.predict(X_val)
f1_score(y_val, y_pred, average='weighted')

About 65%... Pretty good!

### Neural network

The network we're going to train wants a different shape of array. It needs a 4D array of shape `[instances, width, height, channels]`. So for us this will be `[528, 32, 32, 1]`. 

In [ ]:
X_train = X.reshape(X.shape[0], int(np.sqrt(X.shape[1])), int(np.sqrt(X.shape[1])), 1) #channel last
X_test = X_val.reshape(X_val.shape[0], int(np.sqrt(X_val.shape[1])), int(np.sqrt(X_val.shape[1])), 1)

<div class="alert alert-success">
Most neural network algorithms expect the target variables to be "one hot encoded". This means, a 1D vector for each label with a dimension of the number of classes. So, for our case with 4 distinct classes, `y[0] = array([1, 0, 0, 0])`, if `y[0]` belongs to the first class. 

**Exercise:**
* Scikit-learn has encoders that can deal with this process automatically. Find the right encoder to produce the expected one hot encoded target vector.
</div>

We're ready to setup a network. We'll add layers to a `Sequential` keras network and we'll make use of a `scikit-learn` compatible `KerasClassifier` to build our training pipeline. First, the appropriate imports:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import joblib

To make life easy, we can define a function that buils the model. Inside this function there will be a collection of layers, activations and settings for each one.

In [ ]:
num_classes = 3

img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 1)


def make_model(kernel_size=3, pool_size=2, filters=16, dense_layer_sizes=[16]):
    model = Sequential()
    
    # Convolutional layers.
    model.add(Conv2D(filters, kernel_size, padding='valid', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(filters, kernel_size))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    
    # Dense layers.
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
        model.add(Activation('relu'))    
    model.add(Dropout(0.5))

    # Output layer.
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

Here's what the model will look like with all the default arguments; it has nearly 53,000 parameters.

In [ ]:
model = make_model()

model.summary()

Just as in the `scikit-learn` example, we can make use of `GridSearchCV` to setup a training run that can validate itself. This is just an example of that:

In [ ]:
dense_size_candidates = [[32], [16, 16]]  # Try a single Dense, and try two Dense layers

kclf = KerasClassifier(make_model, batch_size=32)
clf = GridSearchCV(kclf,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # epochs is avail for tuning even when not
                                     # an argument to model building function
                                     'epochs': [3],
                                     'filters': [8],
                                     'kernel_size': [3],
                                     'pool_size': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1, cv=2)
clf.fit(X_train, y_train)

We can quickly look at some of the training metrics from the classification task:

In [ ]:
# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = clf.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(X_test, y_test) # included in keras
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

A couple of thins about predictiong from the `KerasClassifier` and the keras model directly:
- The `scikit-learn` `keras` classifier doesn't return a one hot encoded prediction. It returns the index of class where the probability has a maximum value.
- The `keras` model itself does return the `softmax` output: a 1D vector per image with the probability of each class membership.

In [ ]:
clf.predict(X_test)

In [ ]:
np.array([np.where(r==1)[0][0] for r in y_test])

### Class probability

The network can emit probabilities. Each instance's vector contains the probability of each class. The argmax of this gives the predicted class.

In our poor result, the classes are almost equally likely.

In [ ]:
y_probs = clf.predict_proba(X_val.reshape(-1, 32, 32, 1))
y_probs[:10]

The `utils` module contains a utility for visualizing the results. The predicted class is shown with the prediction probability; underneath, the actual class is shown in brackets.

In [ ]:
import utils

utils.visualize(X_val, y_val, y_probs, ncols=5, nrows=3, shape=(32, 32))
plt.show()

We can still use the usual `scikit-learn` metrics to check the quality of the model:

In [ ]:
from sklearn.metrics import f1_score

f1_score(np.array([np.where(r==1)[0][0] for r in y_test]), clf.predict(X_test), average='weighted')

The same idea of model persistance applies to `keras` models via `joblib`.

In [ ]:
clf.best_estimator_.model.save('fossil_model.h5')

### Cloud power!

Our network doesn't perform very well. Maybe more filters, more layers, or more training will help. Unfortunately, we'll quickly hit the ceiling of performance on our laptops. We need a bigger computer!

Luckily, Google lets us use their computers, and even their GPUs and TPUs, for free in Google Colab.

Here's a TensorFlow CNN running on Google Colab:

### https://ageo.co/evRsvJ

*Remember to change the Runtime to TPU.*

# Transfer Learning application

We'll use `mobilenet` a popular image classification NN architecture to help classify our fossil images we'll use the weights trained by someone else!

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We'll initialize a `MobileNet` model and we'll remove the last layer. In this way we can append our own last layer and only train that with our fossil images. Note that we're using a different way to "add" or connect layers. We're using the *functional* way of interacting with the Keras API:

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) # We add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024, activation='relu')(x) # Dense layer 2
x = Dense(512, activation='relu')(x)  # Dense layer 3
preds = Dense(4, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

We can specify how many layers we want to freeze and how many we want the weights to be recalculated.

In [ ]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

We can also make use of `keras` included ability to process images for us. This will always save a significant amount of time.

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator = train_datagen.flow_from_directory('../data/fossils/mobilenet/train/',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = train_datagen.flow_from_directory('../data/fossils/mobilenet/val/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=77,
                                                  class_mode='categorical',
                                                  shuffle=False)

Finally, we just need to re-train the network given the setup specified.

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=val_generator,
                    validation_steps=1,
                    epochs=10)

Let's load an image and make a classification using a our model:

In [ ]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [ ]:
new_image = load_image('../data/fossils/mobilenet/val/ammonites/0076.jpg', True)

In [ ]:
np.argmax(model.predict(new_image))

In [ ]:
val_generator.class_indices

# What if we just want to use a pre-trained model already trained:

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg19 import decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
# load the model
model = VGG19()
# load an image from file
image = load_img('../data/fossils/mobilenet/val/ammonites/0076.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print(f'{label[1]} {label[2]*100:.2f}')